# Create the grid for Richards 1D

This code allows to create a mesh for 1D PDE problem:
    - domain discretization
    - parameters setting
    - initial condition setting
All output data are stored in a NetCDF file

    -Author: Niccolò Tubini and Riccardo Rigon
    -License: Creative Commons 4.0

In [11]:
%%capture
%run RichardsMeshGenKosugi.py

In [12]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from netCDF4 import Dataset

# Standard imports 
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.layouts import gridplot
from bokeh.models.widgets import Panel, Tabs
from bokeh.io import output_file, show
from bokeh.plotting import figure
from bokeh.models import BoxSelectTool
from bokeh.models import HoverTool

output_notebook()

Loading BokehJS ...

## Define some variables:
- folderPath: path of the fodlder containing the input file (.csv) -> your_path/data/RichardMeshGen_input
- inFileName: name of the input file (.csv)


- outFileName: name of the output file (.nc) with the entire file path -> your_path/data/Grid_NetCDF/grid.nc
- outputTitle: string containing a title for the output, it is saved within the file
- outputSummary: string containing a description for the output, it is saved within the file
- outputDate: string containing the date of file creation
- outputInstitution: string containing the name of whom created the file

In [13]:
folderPath = "C:/Users/Niccolo/OMS/OMS_Project_Richards1D/data/RichardMeshGen_input"

inputFileName = "Clay_noPonding_Kosugi.csv"

outputFileName =  "C:/Users/Niccolo/OMS/OMS_Project_Richards1D/data/Grid_NetCDF/Clay_noPonding_Kosugi_TEST_TEST.nc"

outputTitle = '''Grid 1 layer clay no ponding water, constant psi with 0m at bottom, 
            clay thetaS =  0.35, thetaR = 0.07, Ks = 0.00000098m/s, alphaSpecificStorage = 0, betaSpecificStorage = 0
                  Kosugi model: r = 0.000043m, sigma = 0.6m'''
outputSummary = ''

outputDate = '24/05/2018'

outputInstitution = 'Geoframe'

## Define the type of intial condition for water suction $\psi$:
Possible choices are:

    -'hydrostatic'
    -'constant'
    -'linear interpolation'

In [14]:
icType = 'hydrostatic'

## Set some plot features

In [15]:
labelSize = 18
titleSize = 22
legendSize = 14
axisTicksSize = 14

lineWidth =1.5

In [16]:
os.chdir(folderPath)
os.listdir()

['Casulli2010_test2.csv',
 'Casulli2010_test2VG.csv',
 'ClaySand_noPonding.csv',
 'ClaySand_noPonding.PNG',
 'Clay_noPonding.csv',
 'Clay_noPonding.PNG',
 'Clay_noPonding_BC.csv',
 'Clay_noPonding_Dry.csv',
 'Clay_noPonding_Kosugi.csv',
 'SandClay_01Ponding_Measurement.csv',
 'SandClay_01Ponding_Measurement.PNG',
 'Sand_01Ponding.csv',
 'Sand_01Ponding.PNG',
 'SiltSand_noPonding_Dunne.csv']

In [17]:
data = pd.read_csv(inputFileName)
#print(data.head())
#print(data.index)
#print(data.columns)
data

,Type,eta,N,psi,thetaS,thetaR,Ks,r,sigma,alphaSpecificStorage,betaSpecificStorage,et
0,L,0,320,-2,0.35,0.07,0.000098,0.000043,0.6,0,0,0
1,L,-2,-999,0,-999.00,-999.00,-999.000000,-999.000000,-999.0,-999,-999,-999


### This section computes geometrical quantities of the discretized domain
For an explanantion of the data in the above dataFrame, please read first the presentation at 

### From the data in the csv file, the main numpy array are built:
    -eta: vertical coordinate of control volume centroids, positive upward with origin set at soil surface [m]
    -etaDual: vertical coordinate of control volume interfaces, positive upward with origin set at soil surface [m]
    -deltaZ: length of each control volume [m]
    -spaceDelta: distance between two adjacent control volume. It is used to compute gradients [m]
    -z: vertical coordinate of control volume centroids, positive upward with origin set at soil column bottom [m]
    -zDual: vertical coordinate of control volume interfaces, positive upward with origin set at soil column bottom [m]

In [18]:
[eta,etaDual,deltaZ,spaceDelta,z,zDual] = buildData(data)

In [19]:
showMesh(data,z,eta,etaDual,labelSize,titleSize,legendSize,axisTicksSize,lineWidth)

### From the data in the csv file, the initial condition for water suction $\psi$ is defined
    -hydrostatic distribution for a given $\psi$ @bottom -> 'hydrostatic'
    -constant distribution for a given $\psi$ @bottom -> 'constant'
    -linear interpolation from measurement points -> 'linear interpolation'

In [9]:
psiIC = setInitialCondition(data,eta,z,icType)

In [10]:
showInitialCondition(psiIC,eta,icType,labelSize,titleSize,legendSize,axisTicksSize,lineWidth)

In [11]:
showTotalHead(psiIC,z,eta,icType,labelSize,titleSize,legendSize,axisTicksSize,lineWidth)

### Assign hydraulic properties to each control volume of the grid
For an explanantion of the data in the above dataFrame, please read first the presentation at 

In [12]:
[thetaS, thetaR, Ks, r, sigma, alphaSpecificStorage,betaSpecificStorage, et] = setParameters(data,eta)

In [13]:
showParameters(thetaS,thetaR,Ks,r,sigma,alphaSpecificStorage,betaSpecificStorage,et,eta,labelSize,titleSize,axisTicksSize,lineWidth)


## Save all grid data in a NetCDF file


In [14]:
writeGridNetCDF(eta,etaDual,z,zDual,spaceDelta,deltaZ,psiIC,thetaS,thetaR,Ks,r,sigma,alphaSpecificStorage,betaSpecificStorage,et,outputFileName,outputTitle,outputInstitution,outputSummary,folderPath,outputDate,inputFileName)

*** SUCCESS writing!  C:/Users/Niccolo/OMS/OMS_Project_Richards1D/data/Grid_NetCDF/Clay_noPonding_Kosugi_TEST_TEST.nc
